## Imports


In [116]:
from langchain_community.chat_models import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.tools import DuckDuckGoSearchRun

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_openai import OpenAIEmbeddings

from langchain.tools.retriever import create_retriever_tool

from langchain.utils.math import cosine_similarity

from openai import OpenAI
from dotenv import load_dotenv


## Pegando .env api

In [117]:
load_dotenv() 
client = OpenAI()

## loadando modelo

In [118]:
from langchain.agents import create_openai_functions_agent, AgentExecutor

model = ChatOpenAI(
    model='gpt-3.5-turbo-1106',
    temperature=0.7
)

## loaders e embedding

In [119]:

loader_web = WebBaseLoader("https://www.metro.sp.gov.br/")
docs = loader_web.load()
    
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20
)
splitDocs = splitter.split_documents(docs)

embedding = OpenAIEmbeddings()
vectorStore = FAISS.from_documents(docs, embedding=embedding)
retriever = vectorStore.as_retriever(search_kwargs={"k": 3})


## Templates

In [120]:
agente_rotas_template = """Você é um especialista no sistema de Metrô de São Paulo, com foco em determinar as melhores rotas para os passageiros. \
      Você tem amplo conhecimento sobre as linhas do metrô, suas interligações e horários de pico, fornecendo alternativas rápidas e eficientes para quem utiliza o metrô.
"""

agente_horarios_template = """Você é um especialista em atualizações sobre o sistema de Metrô de São Paulo. \
    Você informa sobre greves, interrupções e atrasos, além de conhecer detalhadamente os horários de operação do metrô. \
        Você sugere opções alternativas para os passageiros durante períodos de interrupção do serviço.

"""

agente_politicas_template = """Você é um especialista em políticas públicas relacionadas ao sistema de Metrô de São Paulo. \
    Você tem conhecimento detalhado sobre as leis, regulamentos e iniciativas que afetam o metrô, e oferece respostas claras \
        e baseadas em fatos sobre como essas políticas impactam os passageiros.
"""



## Exemplos de perguntas pra semantic routing

In [121]:
rotas_perguntas = [
    "Qual é a rota mais rápida da estação Sé até a estação Vila Madalena?",
    "Como posso evitar as estações mais lotadas durante o horário de pico?",
    "Quais são as melhores conexões entre as linhas azul e verde do metrô?",
]

horarios_perguntas = [
    "Há alguma greve no Metrô de São Paulo programada para esta semana?",
    "O metrô está funcionando normalmente durante o feriado?",
    "Quais são os horários de operação do metrô aos domingos?",
]

politicas_perguntas = [
    "Quais são as políticas de acessibilidade nas estações do Metrô de São Paulo?",
    "Como as tarifas do metrô são decididas e ajustadas?",
    "Quais são as políticas de sustentabilidade adotadas pelo Metrô de São Paulo?",
]


## Fazendo embeddings com os exemplos pra calcular qual o cosseno e poder classificar a dúvida

In [122]:
embeddings = OpenAIEmbeddings()

rotas_perguntas_embeddings = embeddings.embed_documents(rotas_perguntas)
horarios_perguntas_embeddings = embeddings.embed_documents(horarios_perguntas)
politicas_perguntas_embeddings = embeddings.embed_documents(politicas_perguntas)

## LLM routing com busca semântica

In [123]:
def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])

    rotas_similarity = cosine_similarity([query_embedding], rotas_perguntas_embeddings)[0]

    horarios_similarity = cosine_similarity([query_embedding], horarios_perguntas_embeddings)[0]

    politicas_similarity = cosine_similarity([query_embedding], politicas_perguntas_embeddings)[0]

    max_similarity = max(
        max(rotas_similarity), max(horarios_similarity), max(politicas_similarity)
    )

    if max_similarity == max(rotas_similarity):
        prompt_escolhido = "rotas"
    elif max_similarity == max(horarios_similarity):
        prompt_escolhido = "horarios"
    else:
        prompt_escolhido = "politicas"
    return prompt_escolhido
input_query = {"query": "Quando o metrô abre e fecha nos finds?"}
prompt = prompt_router(input_query)

## Tool do duck duck go

In [124]:
search = DuckDuckGoSearchRun()
retriever_tools = create_retriever_tool(
    retriever,
    "metro_sp_search",
    "Use o campo de busca para encontrar informações sobre o Metrô de São Paulo.",
)
tools = [search, retriever_tools]

## gerando prmpt a depender do output do routing

In [125]:
def gerar_prompt(prompt_escolhido):
    templates = {
        "rotas": agente_rotas_template,
        "horarios": agente_horarios_template,
        "default": agente_politicas_template
    }

    template = templates.get(prompt_escolhido, templates["default"])

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", template),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )

    return prompt


## Definindo pergunta

In [134]:
pergunta = str(input("Digite sua pergunta: "))

In [135]:
def gerar_prompt(prompt_escolhido):
    templates = {
        "rotas": agente_rotas_template,
        "horarios": agente_horarios_template,
        "default": agente_politicas_template
    }

    template = templates.get(prompt_escolhido, templates["default"])

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", template),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )

    return prompt


In [136]:
from langchain.agents import AgentExecutor
from langchain.agents import create_openai_tools_agent
prompt = gerar_prompt(prompt_router({"query": pergunta}))

from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(model, tools, prompt)
agent_executer = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executer.invoke({"input":pergunta})





> Entering new AgentExecutor chain...

Invoking: `metro_sp_search` with `{'query': 'como ir para a estacao luz da estacao fradique coutinho'}`












Metrô | Cia. do Metropolitano de São Paulo





























 





Skip to main content











SP + Digital







/governosp













SP + Digital







/governosp







 




















Canais de Relacionamento
 








Sua Viagem Mapa da Rede
Linhas e Estações
Acessibilidade e Atendimento Preferencial
Veja mais…


Expansão e Obras Andamentos das Obras
Canal de Relacionamento
Linha 2-Verde
Linha 15-Prata
Linha 17-Ouro
Veja mais…


Negócios
Governança
Transparência
Institucional
Investidores
 
















Para onde você vai?Saiba como chegar ao seu destino






Direto do Metrô






Declaração de Ocorrências
Foi afetado por alguma ocorrência nas linhas operadas pelo Metrô?
        Emita sua declaração.


Ver declarações disponíveis




Canais de Relacionamento

SMS Segurança
11 97333 2252

Informe 

{'input': 'como faço pra ir pra luz da fradqiue coutinho?',
 'output': 'Parece que não obtivemos resultados relevantes para a sua busca. Infelizmente, não conseguimos encontrar a informação que você procura. Você pode me fornecer mais detalhes sobre a sua viagem? Dessa forma, poderei ajudá-lo de forma mais eficaz.'}